code by Jakob

In [ ]:
#! pip install requests

In [ ]:
import pandas as pd
import requests 
import time
import logging

In [2]:
data_dir = "data/"
dataframe_dir = data_dir + "dataframes/"
speeches_dir = data_dir + "speeches/"
speeches_pdf_dir = speeches_dir + "pdf/"
current_metadata_step = 'metadata_s02'

In [3]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
crawl_delay = 5 # from robots.txt
log.info(f"log with crawl delay {crawl_delay}s")

INFO:__main__:log with crawl delay 5s


In [159]:
def create_session():
    session = requests.Session()
    session.headers.update({
        "User-Agent": "speechcor",
        "Connection": "keep-alive",
    })
    log.info('Session created')
    return session

# Download the pdfs

In [ ]:
metadata = pd.read_csv(f'{dataframe_dir}{current_metadata_step}.csv')
metadata.head(1)

,record_id,speaker,speaker_organization,document_symbol,date,sg_number,year,month,day,body,...,doc_number,doc_type,add_part,file_prefix,file_name_pdf,record_id_document,document_symbol_found,lang_field,doc_url,date_document
0,4090177,"Guterres, António, 1949-",UN. Secretary-General,S/PV.9988,2025-08-28,9,2025,8,28,S,...,NaN,PV.9988,NaN,S_2025_PV.9988_,S_2025_PV.9988_speeches.pdf,4089910.0,S/PV.9988,English,https://digitallibrary.un.org/record/4089910/f...,2025-08-28


In [361]:
def download_pdf(session, failed_doc_requests, doc_url, document_symbol, file_name_pdf, crawl_delay=crawl_delay):
    try:
        # pause between requests
        time.sleep(crawl_delay)

        response = session.get(doc_url)

        if response.status_code != 200:
            log.error(f"Web page returned status code: {response.status_code} for search of {document_symbol}")
            failed_doc_requests.append(doc_url)
            return

        # Write content in pdf file
        pdf = open(f"{speeches_pdf_dir}{file_name_pdf}", 'wb')
        pdf.write(response.content)
        pdf.close()
        log.info(f"Downloaded file: {document_symbol} to {file_name_pdf}")
    except Exception as e:
        log.error(f"When processing: {document_symbol} with url {doc_url} last status code: {response.status_code} ")
        log.exception(e)
        failed_doc_requests.append(doc_url)
    # finally:
    #     pdf.close()

In [ ]:
failed_doc_requests = []
session = create_session()

for doc_record in metadata.itertuples():
    download_pdf(session, 
                 failed_doc_requests, 
                 doc_record.doc_url, 
                 doc_record.document_symbol,
                 doc_record.file_name_pdf,
                 crawl_delay=crawl_delay*2)
    if len(failed_doc_requests) > 60:
        break

log.info("All PDF files downloaded")

INFO:__main__:Session created
INFO:__main__:Downloaded file: S/PV.9988 to S_2025_PV.9988_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9962 to S_2025_PV.9962_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9941 to S_2025_PV.9941_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9939 to S_2025_PV.9939_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9938 to S_2025_PV.9938_speeches.pdf
INFO:__main__:Downloaded file: A/79/PV.72 to A_2025_79_PV.72_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9919 to S_2025_PV.9919_speeches.pdf
INFO:__main__:Downloaded file: A/79/PV.67 to A_2025_79_PV.67_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9907 to S_2025_PV.9907_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9884 to S_2025_PV.9884_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9861 to S_2025_PV.9861_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9841 to S_2025_PV.9841_speeches.pdf
INFO:__main__:Downloaded file: S/PV.9821 to S_2024_PV.9821_speeches.pdf
INFO:__main__:Downloaded file: